In [2]:
import pandas as pd
import numpy as np
from groq import Groq
import os
import pinecone
import re
import sys
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import display, HTML

In [ ]:


groq_api_key = 'add-your-api-key'
client = Groq(api_key = groq_api_key)
model = "llama3-70b-8192" #llama 3


In [ ]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import login



# Login to Hugging Face
login(token="add-your-key")

# Using Meta-Llama model with a specified tokenizer

model_id = "meta-llama/Meta-Llama-3-70B-Instruct" 

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Function to calculate the token length
def token_len(text):
    tokens = tokenizer.encode(text)#
    return len(tokens)

# Embedding each chunk using a SentenceTransformer model
from sentence_transformers import SentenceTransformer
embedding_function = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2") #full



In [5]:
def remove_links(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Remove URLs from the text
    text_without_links = re.sub(url_pattern, '', text)

    return text_without_links

def remove_line_breaks_and_links(text):
    # Remove line breaks
    text_without_line_breaks = re.sub(r'\n{2,}', '\n', text)
    remove_leading_indent = re.sub(r'\n\s+', '\n', text_without_line_breaks)  # Remove leading indentation
    
    # Remove links
    text_without_links = remove_links(remove_leading_indent)

    return text_without_links

In [ ]:
# #Gender
df = pd.read_csv('/Data/target_gender_ads.csv', index_col=False) #gender


#Age
#df = pd.read_csv('/Data/target_age_ads.csv', index_col=False) #age

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.reset_index(drop=True)
print(df.columns, df.shape)

In [ ]:
 # Initialize an empty list to store the assistant's replies
assistant_replies = []

In [ ]:
def chat_completion(client, model, user_question):
    try:
        chat_completion = client.chat.completions.create(
            messages = [
                
                {
                    "role": "user",
                    "content":  user_question ,
                }
            ],
            model = model
        )
        
        response = chat_completion.choices[0].message.content
        return response

    except groq.RateLimitError as e :
        # Handle rate limit exceeded error by waiting for a fixed time (e.g., 60 seconds)
        print("Rate limit exceeded. Waiting for 10 seconds...")
        time.sleep(10)
        return chat_completion(client, model, user_question)  # Retry the request after waiting



In [ ]:
for index, row in df.iterrows():
    text = row['ad_creative_body']
    cleaned_text = remove_line_breaks_and_links(text).strip()
    
    #gender
    user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on gender: male vs female by the provided ad. Choose the most probable option from the gender (either male or female). Don't generate neutral or both gender answers. Provide a brief explanation with reasoning in 20 words. \n Gender: ? \n Explanation: ?"
   #age group
    #user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on the following four age groups: Young adults (18-24), Early working age (25-44), Late working age (45-64), and Senior citizens (65+) by the provided ad. Choose the most probable option from the age groups. Don't generate neutral or multiple age group answers. Provide a brief explanation with the reasoning in 20 words.  \n Age group: ? \n Explanation: ?"
    
    
    assistant_reply = chat_completion(client, model, user_message)
    
    
    # Append the assistant's reply to the list
    assistant_replies.append(assistant_reply)
    
# Add the assistant's replies to the DataFrame
df['llama_reply'] = assistant_replies


In [10]:
df

,id,ad_creative_body,funding_entity,region_distribution,demographic_distribution,impressions,spend,ad_delivery_start_time,assistant_reply,true_gender,predicted_gender,llama_reply
0,4.374617e+14,Achieving our goal of planting 10 million tree...,"CHESAPEAKE BAY FOUNDATION, INC., THE",NaN,"[{'percentage': '0.425676', 'age': '55-64', 'g...","{'lower_bound': '0', 'upper_bound': '999'}","{'lower_bound': '0', 'upper_bound': '99'}",2021-05-27,Gender: Male \nExplanation: Men may be more l...,male,male,Gender: Male \nExplanation: Landowners are oft...
1,2.816868e+15,"We asked some people to try our new beer, and ...","NEW BELGIUM BREWING COMPANY, INC.",NaN,"[{'percentage': '0.175696', 'age': '18-24', 'g...","{'lower_bound': '500000', 'upper_bound': '5999...","{'lower_bound': '7000', 'upper_bound': '7999'}",2021-04-23,Gender: Male\nExplanation: Beer advertisements...,male,male,Gender: Male \nExplanation: Beer and 'Torched ...
2,2.892611e+15,"A new beer, from a future we hope doesn’t exis...","NEW BELGIUM BREWING COMPANY, INC.",NaN,"[{'percentage': '0.239648', 'age': '18-24', 'g...","{'lower_bound': '175000', 'upper_bound': '1999...","{'lower_bound': '1500', 'upper_bound': '1999'}",2021-04-20,Gender: Male \nExplanation: The ad focuses on...,male,male,Gender: Male \nExplanation: Beer marketing and...
3,6.883582e+14,The turbines that provide clean wind energy al...,Dominion Energy,NaN,"[{'percentage': '0.220662', 'age': '45-54', 'g...","{'lower_bound': '3000', 'upper_bound': '3999'}","{'lower_bound': '0', 'upper_bound': '99'}",2021-11-04,Gender: Male \nExplanation: Men are often mor...,male,male,Gender: Female\nExplanation: Females tend to p...
4,1.063178e+15,Meet the first carbon negative fragrance. Air ...,Air Company,NaN,"[{'percentage': '0.616692', 'age': '18-24', 'g...","{'lower_bound': '1000', 'upper_bound': '1999'}","{'lower_bound': '0', 'upper_bound': '99'}",2021-10-22,Gender: Female\nExplanation: The fragrance not...,male,female,Gender: Female \nExplanation: Notes like powde...
...,...,...,...,...,...,...,...,...,...,...,...,...
101,1.076969e+15,More than one million moms are protecting thei...,Moms Clean Air Force,NaN,"[{'percentage': '0.031592', 'age': '18-24', 'g...","{'lower_bound': '2000', 'upper_bound': '2999'}","{'lower_bound': '0', 'upper_bound': '99'}",2022-01-27,Gender: Female \nExplanation: The ad specific...,female,female,Gender: Female \nExplanation: The ad specifica...
102,6.217283e+14,"“In summary, the Act will increase incentives ...",ClimateXChange Education and Research,NaN,"[{'percentage': '0.038462', 'age': '55-64', 'g...","{'lower_bound': '0', 'upper_bound': '999'}","{'lower_bound': '0', 'upper_bound': '99'}",2022-01-21,Gender: Female\nExplanation: The ad's emphasis...,female,female,"Gender: Male \nExplanation: Academic title ""Dr..."
103,5.012463e+14,Things are stalled in DC but Massachusetts can...,ClimateXChange Education and Research,NaN,"[{'percentage': '0.058471', 'age': '65+', 'gen...","{'lower_bound': '2000', 'upper_bound': '2999'}","{'lower_bound': '0', 'upper_bound': '99'}",2022-01-20,Gender: Female \nExplanation: Women typically...,female,female,Gender: Female \nExplanation: Research shows f...
104,2.865347e+14,"$𝟕𝟔𝟖,𝟎𝟎𝟎,𝟎𝟎𝟎,𝟎𝟎𝟎\n \n\n That’s how much Congre...",PRAMILA FOR CONGRESS,NaN,"[{'percentage': '0.051345', 'age': '45-54', 'g...","{'lower_bound': '0', 'upper_bound': '999'}","{'lower_bound': '0', 'upper_bound': '99'}",2022-01-19,Gender: Female \nExplanation: Emphasizing soc...,female,female,Gender: Female\nExplanation: The writing style...


In [ ]:
# Print or save the updated DataFrame with assistant's replies
df.to_csv('/Data/gender_pred_llama.csv')